In [3]:
# Import modules to make a connection to DB and query
import pandas as pd
from sqlalchemy import create_engine

In [11]:
# DB credentials
db_username = 'db_username'
db_password = 'db_password'
db_host = 'db_host'
db_port = 'db_port'
db_name = 'db_name'

In [5]:
# Create connection with DB credentials
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# Customer Reviews Table Queries

### Descriptive Query:

#### Business Question: What is the average overall customer satisfaction ratings for each car model?

In [5]:
Cust_Reviews_BQ_sql_query = """
SELECT 
    YEAR(date_posted) AS year,
    CarModel,
    AVG(overall_rating) AS avg_overall_rating,
    AVG(AVG(overall_rating)) OVER (PARTITION BY CarModel ORDER BY YEAR(date_posted)) AS running_avg_overall_rating
FROM 
    CustomerReviews
GROUP BY 
    year, CarModel
ORDER BY 
    CarModel, year;
"""

In [6]:
Cust_Reviews_BQ_sql_results = pd.read_sql_query(Cust_Reviews_BQ_sql_query, con=engine)

In [7]:
print(Cust_Reviews_BQ_sql_results.head())

   year             CarModel  avg_overall_rating  running_avg_overall_rating
0  2021    Chevrolet Bolt EV            5.000000                    5.000000
1  2022    Chevrolet Bolt EV            4.000000                    4.500000
2  2023    Chevrolet Bolt EV            4.562500                    4.520833
3  2024    Chevrolet Bolt EV            3.875000                    4.359375
4  2022  Ford Mustang Mach-E            4.217391                    4.217391


In [20]:
Cust_Reviews_BQ_sql_results.to_csv('Cust_Reviews_BQ_sql_results.txt', sep='|', index=False)

### Insight:

#### Number of customer reviews vary from year to year, so ratings can flucuate dramatically. Descpite this Tesla Model Y has an inital overall rating of 5 in 2020 which then decreases to 4.23 in 2024. Model S increases from 3.38 in 2020 to 3.79 in 2024. The Model 3 has an overall rating of 1 due to one customer review in 2020, then increases to 3.48 in 2023. Realtive to compeitor models Tesla ratings fall short, expeically for the Model 3 and S.

### Recommendation:

#### Investigate the car attributes that customers are complaining about.

### Prediction:

#### Improve on said attribute to increase customer satisfaction.

### Diagnostic Query:

#### Business Question: How do customer satisfaction aspects (comfort, performance, reliability) trend over time for each car model, and which models show the greatest improvement or decline?

In [6]:
Cust_Reviews_DQ_sql_query = """
WITH YearlyRatings AS (
  SELECT
    CarModel,
    YEAR(date_posted) AS Year,
    AVG(overall_rating) AS AvgOverallRating,
    AVG(comfort) AS AvgComfort,
    AVG(performance) AS AvgPerformance,
    AVG(value) AS AvgValue,
    AVG(reliability) AS AvgReliability,
    AVG(interior) AS AvgInterior
  FROM CustomerReviews
  GROUP BY CarModel, YEAR(date_posted)
),
YearlyChanges AS (
  SELECT
    Current.Year,
    Current.CarModel,
    Current.AvgOverallRating,
    Current.AvgComfort,
    Current.AvgPerformance,
    Current.AvgValue,
    Current.AvgReliability,
    Current.AvgInterior,
    Current.AvgOverallRating - COALESCE(Previous.AvgOverallRating, Current.AvgOverallRating) AS OverallRatingChange,
    Current.AvgComfort - COALESCE(Previous.AvgComfort, Current.AvgComfort) AS ComfortChange,
    Current.AvgPerformance - COALESCE(Previous.AvgPerformance, Current.AvgPerformance) AS PerformanceChange,
    Current.AvgValue - COALESCE(Previous.AvgValue, Current.AvgValue) AS ValueChange,
    Current.AvgReliability - COALESCE(Previous.AvgReliability, Current.AvgReliability) AS ReliabilityChange,
    Current.AvgInterior - COALESCE(Previous.AvgInterior, Current.AvgInterior) AS InteriorChange
  FROM YearlyRatings Current
  LEFT JOIN YearlyRatings Previous ON Current.CarModel = Previous.CarModel AND Current.Year = Previous.Year + 1
)
SELECT *
FROM YearlyChanges
ORDER BY CarModel, Year;
"""

In [7]:
Cust_Reviews_DQ_sql_results = pd.read_sql_query(Cust_Reviews_DQ_sql_query, con=engine)

In [8]:
print(Cust_Reviews_DQ_sql_results.head())

   Year             CarModel  AvgOverallRating  AvgComfort  AvgPerformance  \
0  2021    Chevrolet Bolt EV          5.000000    3.875000        4.375000   
1  2022    Chevrolet Bolt EV          4.000000    4.200000        4.100000   
2  2023    Chevrolet Bolt EV          4.562500    4.250000        4.500000   
3  2024    Chevrolet Bolt EV          3.875000    3.625000        3.375000   
4  2022  Ford Mustang Mach-E          4.217391    3.391304        3.521739   

   AvgValue  AvgReliability  AvgInterior  OverallRatingChange  ComfortChange  \
0  4.125000        4.375000     3.875000               0.0000          0.000   
1  4.100000        3.700000     4.300000              -1.0000          0.325   
2  4.250000        4.000000     4.250000               0.5625          0.050   
3  3.500000        3.125000     3.750000              -0.6875         -0.625   
4  3.086957        2.869565     3.391304               0.0000          0.000   

   PerformanceChange  ValueChange  ReliabilityChan

In [9]:
Cust_Reviews_DQ_sql_results.to_csv('Cust_Reviews_DQ_sql_results.txt', sep='|', index=False)

In [10]:
engine.dispose()

### Insight:

#### Tesla Model 3 appears to initially have negative change in ratings in comfort and performance in the year 2021 and 2022 but then sees almost 1 point improvements in all attributes in the year 2023. The Model S appears to initially have improvements in ratings changes across all attributes in the year 2021 with a little over than half point decreases in the year 2022 followed by 1 point plus rating changes in the year 2023. With only one review in 2024 being severily negative. For the model Y in 2021 there was a 1.5 decrease in reliabilty rating followed by marginal decreases in every rating attribute in 2022, with increases in comfort, performance, and value, and decreases in reliability and interiror in the year 2023.

### Recommendation:

#### Improve performance and comfort for Model S and 3, as customers reviews reflect a decrease or stadic change in ratings for these attributes.

### Prediction:

#### Improvement in these attributes of Tesla Models would increase average overall customer satsicafction and lead to increased customer loyalty.